In [1]:
import jax

jax.config.update("jax_enable_x64", val=True)

In [2]:
import jax.numpy as jnp

from qpm import cwes, mgoslt

design_temp = 70.0
b_initial = jnp.array(1.0 + 0.0j)
design_wl = 1.031
# optimized_widths = jnp.load("../datasets/optimized_thg_2000_1489_e4.npy")
# domain_widths = optimized_widths

delta_k1_design = mgoslt.calc_twm_delta_k(design_wl, design_wl, design_temp)
delta_k2_design = mgoslt.calc_twm_delta_k(design_wl, design_wl / 2, design_temp)

# num_domains = 555
# num_domains = domain_widths.shape[0]
shg_width = jnp.pi / delta_k1_design
num_shg_domains = 321
sfg_width = jnp.pi / delta_k2_design
num_sfg_domains = 1168
num_domains = num_shg_domains + num_sfg_domains

shg_widths = jnp.array([shg_width] * num_shg_domains)
sfg_widths = jnp.array([sfg_width] * num_sfg_domains)
domain_widths = jnp.concatenate(
    [
        jnp.full(num_shg_domains, shg_width),
        jnp.full(num_sfg_domains, sfg_width),
    ],
)

kappa_mag = 1.31e-5 / (2 / jnp.pi)
kappa_vals = kappa_mag * ((-1) ** jnp.arange(num_domains))

z_coords, full_trace = cwes.simulate_shg_npda_trace(domain_widths, kappa_vals, delta_k1_design, b_initial)

In [3]:
import plotly.graph_objects as go

shg_intensity = jnp.abs(full_trace)

# Plotlyでグラフを作成・表示
fig = go.Figure(data=go.Scatter(x=z_coords, y=shg_intensity, mode="lines", name="SHG Intensity"))

fig.update_layout(
    title="Second-Harmonic Generation Growth (NPDA Simulation)",
    xaxis_title="Propagation Distance, z (μm)",
    yaxis_title="SHG Intensity, |A₂ω|",
)

fig.show()

In [4]:
from qpm import cwes

# --- Calculate Propagation Trace for Optimized Structure ---
# Calculate phase mismatches at the design wavelength
delta_k1_design = mgoslt.calc_twm_delta_k(design_wl, design_wl, design_temp)
delta_k2_design = mgoslt.calc_twm_delta_k(design_wl, design_wl / 2, design_temp)
b_initial = jnp.array([1.0 + 0.0j, 0.0 + 0.0j, 0.0 + 0.0j])  # Initial amplitudes for FW, SHW, THW
kappa_vals = kappa_mag * jnp.power(-1, jnp.arange(domain_widths.shape[0]))

# Run simulation to get the trace of amplitudes at each domain
_, full_trace = cwes.simulate_twm_with_trace(domain_widths, kappa_vals, delta_k1_design, delta_k2_design, b_initial)

# Calculate the propagation distance z
z = jnp.concatenate([jnp.array([0.0]), jnp.cumsum(domain_widths)])

# Calculate the power for each wave
shw_amp = jnp.abs(full_trace[:, 1])

optimized_widths = jnp.load("../datasets/optimized_thg_2000_1489_e4.npy")
kappa_vals_optimized = kappa_mag * jnp.power(-1, jnp.arange(optimized_widths.shape[0]))
_, full_trace_optimized = cwes.simulate_twm_with_trace(optimized_widths, kappa_vals_optimized, delta_k1_design, delta_k2_design, b_initial)
shw_amp_optimized = jnp.abs(full_trace_optimized[:, 1])
z_optimized = jnp.concatenate([jnp.array([0.0]), jnp.cumsum(optimized_widths)])

# Create the plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=z, y=shw_amp, mode="lines", name="SHW Intensity (Periodical)"))
fig.add_trace(go.Scatter(x=z_optimized, y=shw_amp_optimized, mode="lines", name="SHW Intensity (Optimized)"))

fig.update_layout(
    title="Wave Propagation in Optimized Structure",
    xaxis_title="Propagation Distance z (μm)",
    yaxis_title="Amplitude (a.u.)",
    template="plotly_white",
    legend_title="Wave",
)
fig.show()

In [1]:
from qpm import mgoslt

mgoslt.sellmeier_n_eff(1.031, 70)

W0108 15:27:08.053978   59318 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W0108 15:27:08.058172   59225 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.


Array(2.1325023, dtype=float32)